 # 1. Introduction and Defining Business Problem:

Berlin, It is a place where I am staying right now. In terms of population and area, Berlin is the largest city and capital for Germany. Berlin is the second famous startup hub in Europe.

## Problem Statement: Prospects for opening a food Restaurant or a new office in Berlin.

During the daytime, especially during lunch hours, office areas provide huge opportunities for restaurants.  An average meal priced (one lunch meal 5€). The shops are usually always full during lunch hours (11 am — 2-30 pm. Given this scenario, I will be showing the benefits and pitfalls of opening a restaurant in highly dense office places including office delivery. However, I am unaware of the profit, I do believe there will be huge benefits by opening a restaurant in the dense area of companies. I will be covering the top 10 places in Berlin.

Additionally, by understanding the type of companies that are located in each area will result in valuable information for opening a new office. Such as finding an area relatively less cost for opening an office and public transportation analysis to reach this company.

## Description of the data and solving the problems

To solve the above problems, a location-based dataset is important. However, neighther it is not available directly in internet nor from the Foursquare website. Hence, I decided to scrape the required data. 

There are 2 datasets:
    
    Company-related data with following columns:
        1. Company name: In Germany, a company is characterized as mbH, GmbH, AG, AG &Co.
        2. Address: It is composed of a street name, GPS coordinates, zip code, neighborhood
        3. Category: It is a type of company i.e software company, construction company e.t.c

    Berlin geographical dataset is composed with the following columns:
        1. Zip code
        2. Neighborhood
        3. District



### To solve above problems the following questions needed to answer.


    1. How many companies are located in each area? (i.e per zip code)
    2. Which type of shops are located near to each company? (i.e with the help GPS coordinates of a company). This particular question will be answered through Foursquare data.
    3. How similar each company in terms of its services?
    4. Possible public transportation services to reach the company?
    5. How far the residential area is located? 
    6. What are the average rental prices for a given area? By analyzing clusters of companys that are located for an area. (i.e This question will be answered if the data relevant data is available else will be skipped.)  


As a part of the report, I will walk you through each step of this project and address them separately. These answeres will justify a better place to open a restaurant or to open an office for my stakeholders. 

Let me, show you an outline of the initial data preparation. 



### Target Audience

#### What type of clients or a group of people would be interested in this project?

    1.For data scientists, who wnats to do exploratory data analysis techniques to obtain necessary data, analyze it, and, finally be able to tell a story out of it.
    2.Business personnel who wants to invest or open a restaurant. This analysis will be a comprehensive guide to start or expand restaurants targeting the large pool of office places during lunch hours.
    3. Furthermore, the analysis of company locations in Berlin will hugly benifit the business personnel for opening their new office.

# 2. Preprocessing the dataset  having companies list, postal codes and neighborhoods in Berlin

In [1]:
import wikipedia
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import unicodedata
import requests
import json

### Extract Berlin neighborhoods and boroughs from wiki using BeautifulSoup

#### If like to scrape the data, run the following blocke of code.

```python
wiki_page = "https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins"

html_ = requests.get(wiki_page)
soup = BeautifulSoup(html_.content, "html.parser")
table = soup.find_all("table")
data_table = table[2]

data_nei_ = []
columns = ["Nr","Ortteil","Bezirk","Flaeche","Einwohner_30_07_2019","Einwohner_km2"] 

for tr in data_table.find_all("tr")[1:]:
    data = []
    for td in tr.find_all("td"):
        data.append(td.get_text().strip())
    data_nei_.append(data)
data_nei_
df_nei = pd.DataFrame(data_nei_, columns=columns)
df_nei
```

### Step1: Load dataset having district and its neighborhoods

In [100]:
berlin_neighborhoods = pd.read_csv("data/berlin_places.csv")
berlin_neighborhoods = berlin_neighborhoods[["Ortsteil","Bezirk"]]
berlin_neighborhoods.head()

,Ortsteil,Bezirk
0,Mitte,Mitte
1,Moabit,Mitte
2,Hansaviertel,Mitte
3,Tiergarten,Mitte
4,Wedding,Mitte


### Step2: Load the daset having neighborhood and its postal codes

In [96]:
berlin_postacodes = pd.read_excel("data/Bundesland Berlin.xlsx")
berlin_postacodes[berlin_postacodes.Ortsteil=="Wedding"]

,PLZ,Ortsteil
241,13347,Wedding
243,13349,Wedding
244,13351,Wedding
246,13353,Wedding
249,13357,Wedding
251,13359,Wedding
256,13405,Wedding
259,13407,Wedding
262,13409,Wedding


### Step3: Load the dataset having company and its address, category of comapny, gps coordinates. 

###### Scraping of such data is still in progress and will be updated soon...........

In [289]:
profiles = pd.read_csv("data/company_profile.csv")
profiles = profiles[["url","location","info"]]
profiles['location'] = profiles['location'].apply(lambda x: "{:.3f}".format(x) if not pd.isnull(x) else x)

In [290]:
profiles['location'] = profiles['location'].astype(str)
profiles['info'] = profiles['info'].astype(str)
cleaned_profiles = profiles.groupby(["url"])["info"].agg([('info', ','.join)])
cleaned_location = profiles.groupby(["url"])["location"].agg([('location', ','.join)])
cleaned_profiles["profile"] = cleaned_profiles["info"]+ "," +cleaned_location["location"]

In [291]:
def correct(x):
    cleaned_str = []
    for i in x.split(","):
        if i != 'nan':
            cleaned_str.append(i)
    return cleaned_str

cleaned_profiles["profile"] = cleaned_profiles.profile.apply(lambda x: correct(x))
cleaned_profiles.head()

,info,profile
url,,
(KA) Kraft Automobile GmbH,"nan,nan,(KA) Kraft Automobile GmbH,Wexstrasse ...","[(KA) Kraft Automobile GmbH, Wexstrasse 15, 10..."
(know:bodies) gesellschaft für integrierte kommunikation und bildungsberatung mbh,"nan,nan,(know:bodies) gesellschaft für integri...",[(know:bodies) gesellschaft für integrierte ko...
07schanksysteme gmbh,"nan,nan,07schanksysteme gmbh,Sprengelstrasse 1...","[07schanksysteme gmbh, Sprengelstrasse 15, 133..."
1 Berlin x Hausverwaltung GmbH + Co. KG,"1 Berlin x Hausverwaltung GmbH + Co. KG,Königi...","[1 Berlin x Hausverwaltung GmbH + Co. KG, Köni..."
1 Berlin x Recon GmbH + Co. KG,"1 Berlin x Recon GmbH + Co. KG,Königin-Luise-S...","[1 Berlin x Recon GmbH + Co. KG, Königin-Luise..."


In [295]:
profiles_list = cleaned_profiles["profile"].values.tolist()
company_data = pd.DataFrame(profiles_list)[[0,1,2,3,4,5]]
company_data.columns= ["Name", "Street","Zipcode","City","Lat","Lan"]
print("Shape of data: ",company_data.shape)
company_data[company_data.isnull().any(axis=1)].head()

Shape of data:  (4230, 6)


,Name,Street,Zipcode,City,Lat,Lan
3,1 Berlin x Hausverwaltung GmbH + Co. KG,Königin-Luise-Strasse 43,14195,Berlin,None,None
4,1 Berlin x Recon GmbH + Co. KG,Königin-Luise-Strasse 43,14195,Berlin,None,None
5,1 Berlin x Recon Holding GmbH,Königin-Luise-Strasse 43,14195,Berlin,None,None
12,2 + 1 Global Projects GmbH,Chausseetrasse 14,10115,Berlin,None,None
23,4initia GmbH,Söhtstrasse 6 a,12203,Berlin,None,None


In [300]:
# droping data which does't have Latitude an longitude
company_data = company_data.dropna()
company_data = company_data.reset_index(drop=True)
company_data.head()

,Name,Street,Zipcode,City,Lat,Lan
0,(KA) Kraft Automobile GmbH,Wexstrasse 15,10715,Berlin,52.479,13.336
1,(know:bodies) gesellschaft für integrierte kom...,Sophie-Charlotten-Strasse 103,14059,Berlin,52.518,13.287
2,07schanksysteme gmbh,Sprengelstrasse 15,13353,Berlin,52.542,13.355
3,1-2-3 Gebäudemanagement Berlin GmbH,Fredericiastrasse 28,14059,Berlin,52.511,13.282
4,1000eyes GmbH,Joachimstaler Strasse 12,10719,Berlin,52.503,13.331
